In [2]:
!pip install gTTS --upgrade

In [3]:
!pip install pydub

In [10]:
import io
import re # Import regex for extracting the class number
from pydub import AudioSegment
from gtts import gTTS

# Define the path to the jingle file
jingle_path = "../asset/announcement.mp3"

# Load the jingle file
try:
    jingle_intro = AudioSegment.from_file(jingle_path)
    print("Jingle loaded successfully.")
except Exception as e:
    print(f"Error loading jingle: {e}")
    jingle_intro = None # Handle case where jingle fails to load


# Get text input from the user
user_text = input("Masukkan teks pengumuman: ")

# Create gTTS object
try:
    tts = gTTS(text=user_text, lang='id', slow=False)
    print("gTTS object created.")
except Exception as e:
    print(f"Error creating gTTS object: {e}")
    tts = None


# Save gTTS audio to a BytesIO object and load it as AudioSegment
announcement_audio = None
if tts:
    try:
        audio_stream = io.BytesIO()
        tts.write_to_fp(audio_stream)
        audio_stream.seek(0)
        announcement_audio = AudioSegment.from_file(audio_stream)
        print("Announcement audio created from gTTS.")

        # --- Add logic for rule-based audio panning ---
        # Extract the class number using regex
        match = re.search(r'kelas\s*(\d+)', user_text)
        if match:
            class_number_str = match.group(1)
            try:
                class_number = int(class_number_str)
                print(f"Detected class number: {class_number}")

                # Apply panning based on class number parity
                if class_number % 2 == 0:
                    print("Class number is even. Panning audio to the right.")
                    # Panning to the right (+1.0 is full right)
                    announcement_audio = announcement_audio.pan(1.0)
                else:
                    print("Class number is odd. Panning audio to the left.")
                    # Panning to the left (-1.0 is full left)
                    announcement_audio = announcement_audio.pan(-1.0)
            except ValueError:
                print(f"Could not convert '{class_number_str}' to an integer. No panning applied.")
        else:
            print("No class number found in the input text. No panning applied.")
        # --- End of rule-based audio panning logic ---

    except Exception as e:
        print(f"Error creating announcement audio or applying panning: {e}")

Jingle loaded successfully.
gTTS object created.
Announcement audio created from gTTS.
Detected class number: 9
Class number is odd. Panning audio to the left.


In [11]:
if jingle_intro is not None and announcement_audio is not None:
    jingle_outro = jingle_intro.reverse() # Reverse the jingle for the outro
    combined_audio = jingle_intro + announcement_audio + jingle_outro
    print("Audio combined successfully with reversed outro jingle.")
else:
    print("Cannot combine audio because either jingle or announcement audio failed to load.")

Audio combined successfully with reversed outro jingle.


In [12]:
import IPython.display as ipd
import io

if combined_audio is not None:
    try:
        # Export the combined audio to WAV format in memory
        audio_stream = io.BytesIO()
        combined_audio.export(audio_stream, format='wav')
        audio_stream.seek(0)

        # Play the audio in Colab
        display(ipd.Audio(audio_stream.read(), autoplay=True))
        print("Combined audio is playing.")

    except Exception as e:
        print(f"Error playing audio: {e}")
else:
    print("Cannot play audio because combined_audio is None.")

Combined audio is playing.
